In [1]:
#### Module
import os
import subprocess

import pandas as pd
import optuna

from objective import create_protocol, evaluate

In [2]:
### config
PATH_LOG = os.getcwd() + '/data'
PATH_SCRIPT = os.getcwd() + '/source'

In [3]:
### create study
study = optuna.create_study(storage=f"sqlite:///{PATH_LOG}/geekten.db", study_name="study_turquoise")

[I 2022-09-03 23:43:21,060] A new study created in RDB with name: study_turquoise


In [4]:
### create trial
trial = study.ask()

new_dir = f'/epoch_{trial.number}'
out_dir = PATH_LOG + new_dir
os.mkdir( out_dir )

In [5]:
### execute ot2_protocol.py
create_protocol(trial, PATH_SCRIPT, out_dir)

吸光度データを対象ディレクトリに移す

In [6]:
# execute evaluate.py & get score
score = evaluate(trial, PATH_SCRIPT, out_dir)
print(score)

b'2.501744784094348\n'


In [7]:
study.tell(trial, float(score))

FrozenTrial(number=0, values=[2.501744784094348], datetime_start=datetime.datetime(2022, 9, 3, 23, 43, 22, 501738), datetime_complete=datetime.datetime(2022, 9, 4, 0, 1, 26, 354582), params={'body_red': 147, 'body_green': 0, 'body_blue': 3}, distributions={'body_red': IntDistribution(high=150, log=False, low=0, step=1), 'body_green': IntDistribution(high=3, log=False, low=0, step=1), 'body_blue': IntDistribution(high=3, log=False, low=3, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)

In [8]:
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_body_blue,params_body_green,params_body_red,state
0,0,2.501745,2022-09-03 23:43:22.501738,2022-09-04 00:01:26.354582,0 days 00:18:03.852844,3,0,147,COMPLETE


In [9]:
print(study.best_params)
print(study.best_value)

{'body_blue': 3, 'body_green': 0, 'body_red': 147}
2.501744784094348


In [10]:
csv_dir = PATH_LOG + '/summary' 
os.mkdir( csv_dir )
df.to_csv(csv_dir + f'/epoch_{trial.number}' )